In [1]:
import pandas as pd
import numpy as np
import datetime
pd.set_option('display.max_rows', 500)

In [2]:
funds_unfiltered = pd.read_csv('data/form13f.csv',sep=";")
 # preserve the original dataframe for now 

In [3]:
funds=funds_unfiltered.copy()

In [4]:
print("原始文件基金数量",len(funds["iCIK"].unique()))
# Convert date to Datetime
funds['iFILING_DATE']= pd.to_datetime(funds['iFILING_DATE'])
funds['iPERIOD_END']= pd.to_datetime(funds['iPERIOD_END'])


# Data Prcoessing requirement: Page46

funds = funds[funds['iMARKET_VALUE']!=0]
funds = funds[funds['iQTY'] != 0]
funds = funds.drop_duplicates()
funds = funds[(funds['iFILING_DATE'] >= datetime.datetime(2013, 6, 30)) & (funds['iPERIOD_END'] <= datetime.datetime(2018, 6, 30))]

print("筛完46页Data Processing还是剩基金数量:",len(funds["iCIK"].unique()))



# The fund must complete filings whintin 46d
funds['FILING_INTERVAL'] = funds['iFILING_DATE']-funds['iPERIOD_END']
funds = funds[funds['FILING_INTERVAL']<=datetime.timedelta(days = 45)] 

print("筛完45days 还剩下的基金数量:",len(funds["iCIK"].unique()))


#Fund Selection: Exhibit 3
#  One filing point for each quarter only, no amendament allowed
funds = funds[funds['iAMEND']== 0]
funds = funds[funds['iRESTATEMENT']== 0]
funds = funds[funds['iTYPE']== 0]
funds = funds.drop('iAMEND', axis=1)
funds = funds.drop('iRESTATEMENT', axis=1)
funds = funds.drop('iTYPE', axis=1)

print("筛完修改过的还剩下的基金数量:",len(funds["iCIK"].unique()))



#  The fund must have between 20-200 holdings at least one quarter
temp_df=funds.groupby(["iCIK","iPERIOD_END"]).count()
funds_names_20_200=temp_df[(temp_df["iCUSIP"]>=20)&(temp_df["iCUSIP"]<=200)].index.get_level_values(0).unique()
funds=funds[funds["iCIK"].isin(funds_names_20_200)]

print("筛完20到200只股票范围的还剩下的基金数量:",len(funds["iCIK"].unique()))

#  The fund must be valued between 100MM to 500MM

temp_df=funds.groupby(["iCIK","iPERIOD_END"]).sum()
funds_names_size=temp_df[(temp_df["iMARKET_VALUE"]>=100000000)&(temp_df["iMARKET_VALUE"]<=500000000)].index.get_level_values(0).unique()
funds=funds[funds["iCIK"].isin(funds_names_size)]
print("筛完基金规模还剩下的基金数量:",len(funds["iCIK"].unique()))

#  The fund must have lasted for a year
funds["year"]=funds["iPERIOD_END"].dt.year
temp_df=funds.groupby(["iCIK","year"])["iPERIOD_END"].apply(lambda quarters: quarters.nunique()).to_frame()
funds_names_4Qs = temp_df[temp_df["iPERIOD_END"]>=4].index.get_level_values(0).unique()
funds=funds[funds["iCIK"].isin(funds_names_4Qs)]
print("筛完“至少持续了4个季度”剩下的基金数量:",len(funds["iCIK"].unique()))



原始文件基金数量 9540
筛完46页Data Processing还是剩基金数量: 6034
筛完45days 还剩下的基金数量: 5938
筛完修改过的还剩下的基金数量: 5932
筛完20到200只股票范围的还剩下的基金数量: 3950
筛完基金规模还剩下的基金数量: 2944
筛完“至少持续了4个季度”剩下的基金数量: 1989


Feature X20, X21

In [8]:
prices = pd.read_csv('data/prices.csv',sep=";")
prices['pSP_DATE']= pd.to_datetime(prices['pSP_DATE'])
# Get the prices to calculate market capital of each position
look_up = prices[["pSP_CUSIP","pSP_DATE","pSP_CLOSE"]]
funds=funds.merge(look_up, left_on=["iCUSIP","iPERIOD_END"], right_on=["pSP_CUSIP","pSP_DATE"])
funds.drop(columns=["pSP_CUSIP","pSP_DATE"],inplace=True)
funds["mar_Cap"] = funds["iQTY"] * funds["pSP_CLOSE"]

#Sort the row according to: Funds, Stock, Time
funds.sort_values(by=["iCIK","iCUSIP","iPERIOD_END"],ascending=True,inplace=True)

#Change in Market Capital and Turnover between quarters 
funds["mar_Cap_change"]=funds.groupby(["iCIK","iCUSIP"])["mar_Cap"].shift()
funds["turnover_quart"]=(funds["mar_Cap"]- funds["mar_Cap_change"])/funds["mar_Cap_change"] # (Current Q - previous Q)/Previous Q
funds["mar_Cap_change"] = funds["mar_Cap"]- funds["mar_Cap_change"]

#Change in Quantity between quarters
funds["quantity_change"]=funds.groupby(["iCIK","iCUSIP"])["iQTY"].shift()
funds["quantity_change"] = funds["iQTY"]- funds["quantity_change"]


In [11]:
#  The fund qarterly volatility must be smaller than median of all funds. Exhibit 6 
fund_quater_mar=funds.groupby(["iCIK","iPERIOD_END"])["mar_Cap"].sum().reset_index() # Sum all quarterly mar cap
fund_vol=fund_quater_mar.groupby("iCIK")["mar_Cap"].std() # Get quarterly volatility
fund_vol=fund_vol[fund_vol<fund_vol.median()] # Keep only those that BELOW median average of all funds
funds=funds[funds["iCIK"].isin(fund_vol.index)] # Filter out the names in the Funds Dataframe
print("筛完“波动性小于中位数”剩下的基金数量:",len(funds["iCIK"].unique()))

筛完“波动性小于中位数”剩下的基金数量: 994


In [12]:
#  The fund qarterly turnover must be smaller than median of all funds. Exhibit 6 
fund_turnover= funds.groupby("iCIK")["turnover_quart"].mean()
fund_turnover=fund_turnover[fund_turnover<fund_turnover.median()]
funds=funds[funds["iCIK"].isin(fund_turnover.index)] # Filter out the names in the Funds Dataframe
print("筛完“周转率小于中位数”剩下的基金数量:",len(funds["iCIK"].unique()))

筛完“周转率小于中位数”剩下的基金数量: 497


In [13]:

#reducing "prices" dataframe size by selecting only revelant stocks
prices=prices[prices["pSP_CUSIP"].isin(funds["iCUSIP"])]
#reducing "prices" dataframe size by selecting only revelant dates
prices = prices[(prices['pSP_DATE'] >= datetime.datetime(2013, 6, 30)) & (prices['pSP_DATE'] <= datetime.datetime(2018, 6, 30))]


In [14]:
#Sample result

funds.head(10)

,iRECORD_ID,iCIK,iCUSIP,iPERIOD_END,iFILING_DATE,iQTY,iMARKET_VALUE,iLONG_FRACTION,FILING_INTERVAL,year,pSP_CLOSE,mar_Cap,mar_Cap_change,turnover_quart,quantity_change
408680,40312064,7195,009158106,2013-09-30,2013-10-16,42995.0,4582000.0,0.020734,16 days,2013,99.127,4261965.365,NaN,NaN,NaN
257446,48527318,7195,009158106,2013-12-31,2014-01-21,42995.0,4806000.0,0.019558,21 days,2013,103.973,4470319.135,208353.770,0.048887,0.0
587738,48825019,7195,009158106,2014-03-31,2014-04-14,41995.0,4999000.0,0.020527,14 days,2014,110.726,4649938.370,179619.235,0.040180,-1000.0
218251,49986234,7195,009158106,2014-06-30,2014-07-21,39695.0,5106000.0,0.021350,21 days,2014,119.637,4748990.715,99052.345,0.021302,-2300.0
745988,47090581,7195,009158106,2014-09-30,2014-10-21,39495.0,5141000.0,0.021531,21 days,2014,121.088,4782370.560,33379.845,0.007029,-200.0
58630,50838937,7195,009158106,2014-12-31,2015-01-22,34985.0,5046000.0,0.020751,22 days,2014,134.157,4693482.645,-88887.915,-0.018587,-4510.0
890206,52065643,7195,009158106,2015-03-31,2015-04-27,35110.0,5311000.0,0.021350,27 days,2015,140.714,4940468.540,246985.895,0.052623,125.0
1047860,53960941,7195,009158106,2015-06-30,2015-08-11,34965.0,4784000.0,0.019688,42 days,2015,127.274,4450135.410,-490333.130,-0.099248,-145.0
1215720,54592883,7195,009158106,2015-09-30,2015-11-12,35965.0,4588000.0,0.019983,43 days,2015,118.670,4267966.550,-182168.860,-0.040936,1000.0
1327310,55548779,7195,009158106,2015-12-31,2016-02-08,34930.0,4545000.0,0.019269,39 days,2015,121.023,4227333.390,-40633.160,-0.009520,-1035.0


Feature X16,X17,X18

In [15]:
def annualize_rets(r):
    compounded_growth = (1+r).prod()
    n_periods = r.shape[0]
    return (compounded_growth)**(360/n_periods)-1  # Assume one year has 360 days

In [16]:
# get returns from prices
prices["return"] = prices.groupby("pSP_CUSIP")["pSP_CLOSE"].pct_change()

In [17]:
# Features 16,17,and 18 (slow)
prices["30D"]=prices.groupby(["pSP_CUSIP"],as_index= False)["return"].rolling(window=30).aggregate(annualize_rets)["return"]

In [20]:
prices["60D"]=prices.groupby(["pSP_CUSIP"],as_index= False)["return"].rolling(window=60).aggregate(annualize_rets)["return"]

In [21]:
prices["90D"]=prices.groupby(["pSP_CUSIP"],as_index= False)["return"].rolling(window=90).aggregate(annualize_rets)["return"]

In [36]:
#Test the result
prices[prices["pSP_CUSIP"]=="320734106"].head(100)

,pSP_CUSIP,pSP_EXCHANGE,pSP_TICKER,pSP_DATE,pSP_VOLUME,pSP_OPEN,pSP_HIGH,pSP_LOW,pSP_CLOSE,return,30D,60D,90D
14879,320734106,OS,FLIC,2014-07-25,39553,16.0356,16.2667,15.9556,15.9778,NaN,NaN,NaN,NaN
14880,320734106,OS,FLIC,2014-07-24,27135,16.1867,16.4000,16.0133,16.2178,0.015021,NaN,NaN,NaN
27043,320734106,OS,FLIC,2014-07-23,15444,16.1956,16.2222,16.0400,16.0578,-0.009866,NaN,NaN,NaN
1847927,320734106,OS,FLIC,2014-07-22,37543,16.1111,16.5333,16.1111,16.1378,0.004982,NaN,NaN,NaN
1890819,320734106,OS,FLIC,2014-07-21,17829,16.1333,16.2133,16.1111,16.1156,-0.001376,NaN,NaN,NaN
2241303,320734106,OS,FLIC,2014-07-18,38178,16.2933,16.3333,16.1778,16.2089,0.005789,NaN,NaN,NaN
5690807,320734106,OS,FLIC,2014-07-17,23429,16.5111,16.5733,16.3333,16.3378,0.007952,NaN,NaN,NaN
6817849,320734106,OS,FLIC,2014-07-16,18961,16.7867,16.8756,16.4133,16.5333,0.011966,NaN,NaN,NaN
7148676,320734106,OS,FLIC,2014-07-15,26876,16.3467,16.7689,16.3467,16.6178,0.005111,NaN,NaN,NaN
7356990,320734106,OS,FLIC,2014-07-14,197575,16.8267,17.0280,16.5778,16.6356,0.001071,NaN,NaN,NaN


In [37]:
#Export X16,17,18 to CSV
prices_feature = prices[["pSP_CUSIP","30D","60D","90D"]]

prices_feature.to_csv("prices_X16_X17_X18.csv")

In [39]:
#Export X20,21 to CSV
funds_feature = funds[["iRECORD_ID","iCIK","iCUSIP","mar_Cap_change","quantity_change"]]
funds_feature.to_csv("funds_X20_X21.csv")

In [28]:
funds.shape

(410757, 15)

In [29]:
prices.shape

(6995412, 13)